# Simple Agent Example

In [1]:
%pip install -qU langgraph langchain
%pip install -qU langchain-ollama langchain-community duckduckgo-search
%pip install -qU langchain-text-splitters
from langchain_core.messages.utils import (
    trim_messages,
    count_tokens_approximately
)
import asyncio, random
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.tools import DuckDuckGoSearchRun

from langgraph.checkpoint.memory import MemorySaver
# 'short-term memory
# https://api.python.langchain.com/en/latest/checkpoint/langchain_postgres.checkpoint.PostgresSaver.html for postgres
#

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
memory = MemorySaver()
search = DuckDuckGoSearchRun()
llm = ChatOllama(model="qwen3:1.7b")
agent_prompt = """
    You are a helpful intelligent agent.
"""

max_iterations = 3
recursion_limit = 2 * max_iterations + 1

##########<tools>###



##########</tools>###

# This function will be called every time before the node that calls LLM
# right now, it trims messages when there is not enough memory
# the context length for this model is ~40k, so I went with 20k. Adjust as desired.
def pre_model_hook(state):
    trimmed_messages = trim_messages(
        state["messages"],
        strategy="last",
        token_counter=count_tokens_approximately,
        max_tokens=20000,
        start_on="human",
        end_on=("human", "tool"),
    )
    # You can return updated messages either under `llm_input_messages` or
    # `messages` key (see the note below)
    return {"llm_input_messages": trimmed_messages}

agent = create_react_agent(
    model=llm,
    tools=[search],
    prompt=agent_prompt,
    pre_model_hook=pre_model_hook,
    checkpointer=memory,
)


In [ ]:
## this holds the entire conversation history fed to the agent for now.
messages = []

username = input("User name: ")
greeting = f"Hello {username}. How can I help you today?"
print(greeting)

while True:
    try:
        user_input = input("User: ")
        messages.append(HumanMessage("<sender>"+username+"</sender><message>"+user_input+"</message>"))
        result = agent.invoke({"messages": messages}, {"recursionLimit": recursion_limit, "thread_id": username})
        for msg in result["messages"]:
            msg.pretty_print()
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)

        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break

User name:  ketily


Hello ketily. How can I help you today?


User:  I'm curious what plants are in season in Portland, OR


================================ Human Message =================================

<sender>ketily</sender><message>I'm curious what plants are in season in Portland, OR</message>
================================== Ai Message ==================================

<think>
Okay, the user is asking about plants in season in Portland, OR. I need to figure out what tools to use here. The available tool is duckduckgo_search, which is for current events. So I should perform a search to find out what's happening in Portland's plant seasons.

First, I'll construct the search query. The user mentioned Portland, OR, so the query should include those locations. Maybe something like "plants in season Portland OR 2023" to get the latest information. I'll use that as the query parameter.

I need to make sure the search is specific enough to get relevant results. Since the user is interested in seasonal plants, the search should focus on current seasons. Also, considering the time frame, using 2023 might 